# 自动标注1/200的数据，用于训练高速过滤器

In [1]:
#!AHOCORASICK_BYTES=1 pip install git+https://github.com/WojciechMula/pyahocorasick.git

In [2]:
import os
import json
import random
from matplotlib import pyplot as plt
import numpy as np
from tokenizer import tokenizer,token2str,vocab_size
import torch
import torch.nn as nn
from make_model import make_model
from train_and_use import Batch,CrossEntropyLoss,SimpleAdamOptimizer,OptimizerWrapper,train_server_start
from train_and_use import text_continue
from train_and_use import TOGGLE,STOP
from train_and_use import record
from collections import Counter,defaultdict
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# %matplotlib widget

In [3]:
model = make_model(
    #token是从1开始的，0填充，剩下的用来覆盖全部字节
    vocab_size = vocab_size+1+255,
    embedding_dim = 512,
    key_dim = 256,
    head_number = 8,
    position_information_type = "mask",
    # position_information_type = "sinusoidal",
    # position_information_type = "rotary",
    # position_information_type = "learned",
    enable_affine = True,
    enable_talking_head = True,
    use_diff = False,
    self_attention_block_size = 0,
    feed_forward_dim = 1024,
    enable_layer_norm = True,
    deep = 6,
    dropout_rate = 0.1
).to(device)

In [4]:
model.load_state_dict(torch.load('autotagger_train.weight',weights_only=True))
model = model.eval()

In [5]:
file_high = open('high_auto.txt','w')
file_mid = open('mid_auto.txt','w')
file_low  = open('low_auto.txt','w')

In [6]:
fnames = os.listdir('WuDaoCorpus2.0_base_200G')
np.random.shuffle(fnames)
cnt = 0
for fname in fnames:
    cnt += 1
    print('\r[',cnt,'/',len(fnames),']',end=' '*10)
    with open('WuDaoCorpus2.0_base_200G/'+fname,'r',encoding='utf-8') as f:
        lines = [item['content'] for item in json.load(f)]
        n = len(lines)//200
        n -= n%3
        lines = random.sample(lines,n)
    with torch.amp.autocast("cuda"):
        for i in tqdm(range(0,len(lines),3)):
            tokens_batch = [(tokenizer(lines[i],5.0)+[-255]*1000)[:1000] + tokenizer(' 这段文本的质量按照“高、中、低”三档评价为：',5.0)]
            tokens_batch += [(tokenizer(lines[i+1],5.0)+[-255]*1000)[:1000] + tokenizer(' 这段文本的质量按照“高、中、低”三档评价为：',5.0)]
            tokens_batch += [(tokenizer(lines[i+2],5.0)+[-255]*1000)[:1000] + tokenizer(' 这段文本的质量按照“高、中、低”三档评价为：',5.0)]
            tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
            inputs = torch.from_numpy(tokens_batch).to(device).data
            with torch.no_grad():
                o = text_continue(
                    model,inputs,out_length=1,
                    repeat_penalty_value = 0.0,
                    temperature = 1.0
                )
            for res,line in zip(o,lines[i:i+3]):
                res = token2str(res.cpu().numpy()-255)[-1]
                if res == '高':
                    print(line,file=file_high)
                if res == '中':
                    print(line,file=file_mid)
                if res == '低':
                    print(line,file=file_low)

[ 1 / 366 ]          

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:33<00:00,  4.66it/s]


[ 2 / 366 ]          

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [00:37<00:00,  4.74it/s]


[ 3 / 366 ]          

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 159/159 [00:32<00:00,  4.82it/s]


[ 365 / 366 ]          

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:28<00:00,  4.80it/s]


[ 366 / 366 ]          

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 188/188 [00:38<00:00,  4.85it/s]


In [7]:
file_high.close()
file_mid.close()
file_low.close()